In [ ]:
%%HTML
<style>
.container { width:100% }
</style>

## Funktionen für die Visualisierung

Zur Darstellung der Stadt wird ipycanvas verwendet. Die Label für die informativen Angaben werden von ipywidgets genutzt.

In [ ]:
import ipywidgets
from ipywidgets import Image
import ipycanvas
from ipycanvas import MultiCanvas

Für die Größenverhältnisse in der Darstellung werden einige Konstanten definiert:
* CANVAS_WIDTH = Breite und Höhe der Karte
* BORDER_WIDTH = Breite des Grünstreifens am Rand der Karte
* CELL_WIDTH   = Breite und Höhe der einzelnen Kartenfelder

Anschließend werden die Bilder die die Stadt, den LKW, die Waren und das Ziel symbolisieren geladen

In [ ]:
# Konstanten
CANVAS_WIDTH = 400
BORDER_WIDTH = 20
CELL_WIDTH   = (CANVAS_WIDTH - 2 * BORDER_WIDTH) / 6

# Bilder laden
city  = Image.from_file('img/city.png')
truck = Image.from_file('img/truck.png')
goods = Image.from_file('img/goods.png')
goal  = Image.from_file('img/goal.png')

Die Funktion `init_canvas()` wird zu Beginn aufgerufen und nimmt einen Zustand und stellt diesen Startzustand grafisch dar. Dazu wird eine Leinwand mir vier Ebenen erstellt. Eine für die Karte im Hintergrund, eine für die Zielflagge, eine für den LKW und eine für das Warenpaket. Die Aufteilung auf verschiedene Ebenen hat den Vorteil, das bei der Veränderung von einer Ebene die anderen nicht neu geladen werden müssen. Da sich die Karte und das Ziel nicht verändern, werden die Ebenen 0 und 1 nur am Anfang einmal generiert. So wirkt das Bild stabiler.

Die Positionsnummern von Waren und Ziel werden in Koordinaten und diese schließlich in absolute Positionen auf der Leinwand umgerechnet um danach gezeichnet zu werden.

In [ ]:
def init_canvas(state):
    #canvas[Stadt, Ziel, LKW, Waren]
    canvas = MultiCanvas(4, width = CANVAS_WIDTH, height = CANVAS_WIDTH)
    _, position_goods, position_goal = state
    
    position_goods_label.value = 'Position goods: ' + position_goods_descriptions[position_goods]
    goal_label.value           = 'Goal: ' + stations_descriptions[position_goal]
    
    goal_x_coordinate  = stations[position_goal][1]
    goal_y_coordinate  = stations[position_goal][0]
    goal_x_position  = BORDER_WIDTH + CELL_WIDTH * goal_x_coordinate
    goal_y_position  = BORDER_WIDTH + CELL_WIDTH * goal_y_coordinate 
    
    if position_goods != 4:
        goods_x_coordinate = stations[position_goods][1]
        goods_y_coordinate = stations[position_goods][0]
        goods_x_position = BORDER_WIDTH + CELL_WIDTH * goods_x_coordinate
        goods_y_position = BORDER_WIDTH + CELL_WIDTH * goods_y_coordinate
    
    canvas[0].draw_image(city, 0, 0, CANVAS_WIDTH, CANVAS_WIDTH)
    canvas[1].draw_image(goal,  goal_x_position,  goal_y_position,  CELL_WIDTH, CELL_WIDTH)
    if position_goods != 4:
        canvas[3].draw_image(goods, goods_x_position, goods_y_position, CELL_WIDTH, CELL_WIDTH)
    
    return canvas

Die Funktion `show_start_button()` zeichnet auf dem Stadtplan ein Rechteck, dass das Wort "Start" enthält und optisch einem Button ähnelt.

In [ ]:
def show_start_button(canvas):
    with ipycanvas.hold_canvas(canvas):
        # Hintergrund
        canvas[2].clear()
        canvas[2].fill_style = 'black'
        canvas[2].global_alpha = 0.5
        canvas[2].fill_rect(0, 0, CANVAS_WIDTH, CANVAS_WIDTH)

        # Button
        canvas[2].global_alpha = 1
        canvas[2].shadow_color = 'black'
        canvas[2].shadow_offset_x = 5
        canvas[2].shadow_offset_y = 5
        canvas[2].shadow_blur = 5
        canvas[2].fill_style = 'silver'
        canvas[2].fill_rect(CANVAS_WIDTH/3, (CANVAS_WIDTH/5)*2, CANVAS_WIDTH/3, CANVAS_WIDTH/5)
        
        canvas[2].shadow_offset_x = 0
        canvas[2].shadow_offset_y = 0
        canvas[2].shadow_blur = 0
        
        canvas[2].stroke_style = 'black'
        canvas[2].stroke_rect(CANVAS_WIDTH/3, (CANVAS_WIDTH/5)*2, CANVAS_WIDTH/3, CANVAS_WIDTH/5)
        
        # Text
        canvas[2].fill_style = 'black'
        canvas[2].font = '30px Arial'
        canvas[2].fill_text('Start', CANVAS_WIDTH/2 *0.83, CANVAS_WIDTH/2*1.05)

`visualize_state` zeichnet die Ebenen, die die Waren und den LKW enthalten neu. Wobei die Ebene 3 der Waren nur erneuert wird, falls die Aktion eine Fahrt darstellt, da nur in diesem Fall das Paket tatsächlich bewegt wird.

In [ ]:
def visualize_state(transport):
    # calculating coordinates
    redraw_goods = True
    position_truck, position_goods, position_goal = transport.state
    truck_x_coordinate, truck_y_coordinate = position_truck[1], position_truck[0]
    if position_goods != 4:
        if position_goods == transport.position_goods_old:
            redraw_goods = False
        transport.position_goods_old = position_goods
        goods_x_coordinate, goods_y_coordinate = stations[position_goods][1], stations[position_goods][0]
    else:
        goods_x_coordinate, goods_y_coordinate = truck_x_coordinate, truck_y_coordinate
    
    # calculating positions
    truck_x_position = BORDER_WIDTH + CELL_WIDTH * truck_x_coordinate
    truck_y_position = BORDER_WIDTH + CELL_WIDTH * truck_y_coordinate
    goods_x_position = BORDER_WIDTH + CELL_WIDTH * goods_x_coordinate
    goods_y_position = BORDER_WIDTH + CELL_WIDTH * goods_y_coordinate
    
    # drawing images
    with ipycanvas.hold_canvas(transport.canvas):
        transport.canvas[2].clear()
        transport.canvas[2].draw_image(truck, truck_x_position, truck_y_position, CELL_WIDTH, CELL_WIDTH)
        if redraw_goods:
            position_goods_label.value = 'Position goods: ' + position_goods_descriptions[position_goods]
            transport.canvas[3].clear()
            transport.canvas[3].draw_image(goods, goods_x_position, goods_y_position, CELL_WIDTH, CELL_WIDTH)

Im Folgenden werden Label für die Anzeige der aktuellen Schrittanzahl, des aktuellen Werts, der Position der Waren, des Zielorts, der erfolgten Aktion und die Anzahl der falschen bzw. unmöglichen Aktionen initialisiert.

In [ ]:
step_label           = ipywidgets.Label(value = 'Step: ')
value_label          = ipywidgets.Label(value = 'Current value: ')
position_goods_label = ipywidgets.Label(value = 'Position goods: ')
goal_label           = ipywidgets.Label(value = 'Goal: ')
action_label         = ipywidgets.Label(value = 'Action: ')

Die Funktion `update_canvas()` wird nach jeder Aktion aufgerufen und aktualisiert die Statusanzeigen so wie die grafische Darstellung des aktuellen Zustands.

In [ ]:
def update_canvas(transport):
    
    step_label.value = 'Step: ' + str(transport.steps)
    value_label.value = 'Current value: ' + str(transport.current_value)
    one_below_other = ipywidgets.VBox([step_label, value_label, position_goods_label, goal_label, action_label])
    side_by_side = ipywidgets.HBox([transport.canvas, one_below_other])
    display(side_by_side)
    visualize_state(transport)  
    
    if transport.pause:
        transport.pause = False
        show_start_button(transport.canvas)

`handle_mouse_down()` wird durch Klicken auf die Karte aufgerufen. Die Funktion ruft lediglich `transport_goods()` auf. 

In [ ]:
def handle_mouse_down(self, x, y):
    self.transport_goods()
Transport.handle_mouse_down = handle_mouse_down
del handle_mouse_down